In [11]:
from agents.basic_agents import BasicAgent
from utilities import print_long_text

from config.project_paths import synthetic_data_dir

this notebook holds a function to run rag, evaluation and requerying 

code should be folded into an agent

In [10]:
agent = BasicAgent()

In [12]:
import logging
import re

logging.basicConfig(
    format='%(name)s-%(levelname)s|%(lineno)d:  %(message)s', level=logging.INFO)
log = logging.getLogger(__name__)


def reretrieve__toc(question):
    documents = agent.retriever._get_relevant_documents(question)

    answer = agent.answer_question(
        question, documents)

    evaluation = agent.evaluate_answer(question, answer)
    
    
    match = re.search(r"Score \(1-5\): (\d+)", evaluation.content)

    if match:
        score = match.group(1)
        print(f"Extracted score: {score}")

        try:
            if int(score.strip()) < 4:
                new_answer = agent.regenerate_new_answer_from_evaluation(
                    question, answer, evaluation)
                print('\n\n regenerated answer:')
                print_long_text(new_answer.content)
        except Exception as e:
            print(e)

    else:
        print("Score not found in the string.")

    print('\n\ndocs:')

    for i in documents:
        print(i.metadata)
        print_long_text(i.page_content)

In [18]:
qna_files = list(synthetic_data_dir.rglob('*.csv'))
len(qna_files)

43

In [19]:
from synthetic_data_generation.generate_syn_data import csv_to_list

In [23]:
import random

random_qna_sample = random.sample(qna_files, 15)

qnas = []
for i in random_qna_sample:
    qnas.extend(random.sample(csv_to_list(i), 4))


ValueError: Sample larger than population or is negative

In [26]:
qnas

[['How can you run a simple Java AWT application in Visual Studio Code?',
  'You can run a simple Java AWT application by copying and pasting the provided sample code into a new file in Visual Studio Code.'],
 ['How can you run a JavaFX application in Visual Studio Code?',
  'You can run a JavaFX application by opening the Maven Explorer, expanding `hellofx` > `Plugins` > `javafx`, and then running the Maven goal: `javafx:run`.'],
 ['How can you create a new JavaFX project in Visual Studio Code?',
  'You can create a new JavaFX application by installing the Extension Pack for Java, opening the Command Palette, selecting the command "Java: Create Java Project", and then following the wizard to scaffold a new JavaFX project via Maven Archetype.'],
 ['How can you enable code completions for AWT development in Visual Studio Code?',
  'You can enable completions for AWT development by opening the Command Palette, selecting the command "Java: Help Center", going to the "Student" section, and

In [27]:
from agents.rag_agents import RagAgent


rag_agent = RagAgent()
responses = []

for question, answer in qnas:


    print('question:')

    print(question)
    


    print('\n\nground truth:')
    print(answer)
    

    response = rag_agent.answer_with_rag(question)
    responses.append(response)
    
    print(f'\n\ngenerated answer:{response.answer}')
    print(f'\n\nevaluation score:{response.evaluation_score}')

    
    print('\n\n')

question:
How can you run a simple Java AWT application in Visual Studio Code?


ground truth:
You can run a simple Java AWT application by copying and pasting the provided sample code into a new file in Visual Studio Code.


httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"




generated answer:content='Context is irrelevant to the question.\n\nThe context provided is about creating and running a Java web application in Visual Studio Code, while the question is about running a simple Java AWT (Abstract Window Toolkit) application. These are two different types of Java applications, so the information provided in the context is not relevant to the question.' response_metadata={'token_usage': {'prompt_tokens': 432, 'total_tokens': 500, 'completion_tokens': 68}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-7c1d30d2-3f75-4822-8e5f-575dcf20729e-0'


evaluation score:2



question:
How can you run a JavaFX application in Visual Studio Code?


ground truth:
You can run a JavaFX application by opening the Maven Explorer, expanding `hellofx` > `Plugins` > `javafx`, and then running the Maven goal: `javafx:run`.


httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"




generated answer:content='To run a JavaFX application in Visual Studio Code, you can follow these steps:\n\n1. Install the Extension Pack for Java.\n2. In Visual Studio Code, open the Command Palette and select the command Java: Create Java Project.\n3. Select the option JavaFX in the list, follow the wizard, which will help you scaffold a new JavaFX project via Maven Archetype.\n\nThe debugger extension provides multiple ways to run and debug your Java application.\n\nReason: The context provided is relevant to the question.' response_metadata={'token_usage': {'prompt_tokens': 408, 'total_tokens': 525, 'completion_tokens': 117}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-da6af42d-4b59-44f8-b334-9c8ccb84613f-0'


evaluation score:4



question:
How can you create a new JavaFX project in Visual Studio Code?


ground truth:
You can create a new JavaFX application by installing the Extension Pack for Java, opening the Command Palette, selecting the command "Java: Create 

...



generated answer:content='To disable built-in PHP support in VS Code, you can set the `php.validate.enable` setting to `false` in your User or Workspace Settings. This will turn off PHP linting. If you want to completely remove the PHP language server, you can use the **Disable All Installed Extensions** command in the Command Palette and re-enable the other extensions manually.' response_metadata={'token_usage': {'prompt_tokens': 856, 'total_tokens': 939, 'completion_tokens': 83}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-669baf1f-cfc5-41ed-9fac-60bf2aaab7a4-0'


evaluation score:5



question:
How to change the PHP settings in VS Code?


ground truth:
To change the PHP settings, open your User or Workspace Settings (`kb(workbench.action.openSettings)`) and type 'php' to filter the list of available settings.


httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"




generated answer:content='Reason: The context provided is irrelevant to the question. The information given is about customizing settings in VS Code, creating a data folder for VS Code, and setting environment variables, but it does not mention how to change PHP settings in VS Code.' response_metadata={'token_usage': {'prompt_tokens': 616, 'total_tokens': 670, 'completion_tokens': 54}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-3086c1b0-c2d0-457b-a1d3-f8efbc991c4e-0'


evaluation score:2



question:
What is required to use XDebug for PHP debugging in VS Code?


ground truth:
PHP debugging with **XDebug** is supported through a [PHP Debug extension](https://marketplace.visualstudio.com/items?itemName=xdebug.php-debug). Follow the extension's instructions for configuring **XDebug** to work with VS Code.


httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"




generated answer:content='To use XDebug for PHP debugging in VS Code, you need to follow these steps:\n\n1. Install the PHP Debug extension from the VS Code marketplace.\n2. Configure XDebug to work with VS Code by following the extension\'s instructions.\n3. Set the PHP executable path in VS Code settings if the PHP executable is not on the system path.\n4. Create and set up a `launch.json` file located in your workspace\'s `.vscode` folder for browser debugging configurations.\n\nIf all the context provided is irrelevant, the response would be "Context is irrelevant to the question".\nIf you cannot answer the question confidently, the response would be "I do not know the answer" for the reasons stated above.' response_metadata={'token_usage': {'prompt_tokens': 1163, 'total_tokens': 1331, 'completion_tokens': 168}, 'model': 'mistral-small', 'finish_reason': 'stop'} id='run-51f59f76-c03c-4f50-a973-40742aa7f342-0'


evaluation score:5





In [73]:
from dataclasses import asdict

t = ''
for response, qna in zip(responses, qnas):
    for k,v in asdict(response).items():
        t += k.capitalize() + '\n'
        t += '-'*40 + '\n'
        if k.lower().strip() == 'documents':
            t += '\n\n'.join([i.page_content for i in v]) + '\n\n'
        elif k.lower().strip() == 'evaluation': 
            t += v.content + '\n\n'
        elif k.lower().strip() == 'answer':
            t += v.content + '\n\n'
        else:
            t += str(v) + '\n\n'
        
    t += 'ground truth answer'.capitalize() + '\n'
    t += qna[1] + '\n\n'
    
    t += '='*100
    t += '\n\n'

In [74]:
from config.project_paths import project_root

o = project_root/'synthetic_data_generation'/'trial2.txt'
o.write_text(t)

144962

In [89]:
response = rag_agent.answer_with_rag('implement eks observability')
print_long_text(response.answer.content)

httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
httpx-INFO|1026:  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


To implement EKS observability, you can use the AWS Distro for OpenTelemetry (ADOT) Operator in your Amazon EKS cluster.
This makes it easier for your applications running on EKS to send metric and trace data to monitoring services like
Amazon CloudWatch, Prometheus, and X-Ray. Additionally, you can use Amazon GuardDuty for threat detection in your EKS
clusters, which offers features like EKS Protection and Runtime Monitoring. The GuardDuty service can help protect your
accounts, containers, workloads, and data within your AWS environment. To configure the EKS Pod Identity Agent add-on,
you can use the EKS console and follow the steps provided in the context.


In [91]:
response.evaluation_score

AIMessage(content='Score (1-5): 3\nAnswered: Yes, but could be more specific.\n\nThe answer does provide some information on how to implement EKS observability, but it could be more specific and provide a step-by-step guide or example. The response mentions using the AWS Distro for OpenTelemetry (ADOT) Operator, Amazon CloudWatch, Prometheus, X-Ray, and Amazon GuardDuty for threat detection, but it would be helpful to provide more details or examples on how to use these tools. Additionally, the answer mentions configuring the EKS Pod Identity Agent add-on, but it would be helpful to provide more context or a link to documentation for further reading.', response_metadata={'token_usage': {'prompt_tokens': 382, 'total_tokens': 533, 'completion_tokens': 151}, 'model': 'mistral-small', 'finish_reason': 'stop'}, id='run-2015ae0b-d9ca-49a0-aef0-34571065730f-0')